In [1]:
import os
import requests, json

download files

In [2]:
experiment_list_name = 'file_list_tfchip_hg38_bednarrowpeak.txt'
#os.system('xargs -L 1 curl -O -J -L < ' + experiment_list_name)

In [3]:
experiment_list_temp = open(experiment_list_name).readlines()[1:]
experiment_list = []
for experiment_i in experiment_list_temp:
    experiment_list.append(experiment_i.split('files/')[1].split('/@@')[0])
    
print(experiment_list[0:5])

['ENCFF888ORB', 'ENCFF460GXA', 'ENCFF643AUL', 'ENCFF616GPO', 'ENCFF104VFF']


Preparing a dictionary containing all experiments and the related annotation like genome assembly, cell line, sequencing target.

In [4]:
experiment_summary = {}
headers = {'accept': 'application/json'}
for experiment_i in experiment_list:
    url = 'https://www.encodeproject.org/search/?searchTerm=' + experiment_i + '&frame=object'
    response = requests.get(url, headers=headers)
    experiment_summary[experiment_i] = {}
    for i in range(len(response.json()['@graph'])):
        if('biosample_summary' in response.json()['@graph'][i].keys()):
            experiment_summary[experiment_i]['biobiosample_summary'] = response.json()['@graph'][i]['biosample_summary']
    if('assay_title' in response.json()['@graph'][0]):
        experiment_summary[experiment_i]['assay_title'] = response.json()['@graph'][0]['assay_title']
    if('assembly' in response.json()['@graph'][0]):
        experiment_summary[experiment_i]['assembly'] = response.json()['@graph'][0]['assembly']


In [5]:
for experiment_i in experiment_list[0:5]:
    print((experiment_summary[experiment_i]))

{'biobiosample_summary': 'A549 treated with 100 nM dexamethasone for 2 hours', 'assay_title': 'TF ChIP-seq', 'assembly': 'GRCh38'}
{'biobiosample_summary': 'K562', 'assay_title': 'TF ChIP-seq', 'assembly': 'GRCh38'}
{'biobiosample_summary': 'K562 genetically modified using CRISPR', 'assay_title': 'TF ChIP-seq', 'assembly': 'GRCh38'}
{'biobiosample_summary': 'lower leg skin tissue female adult (53 years)', 'assay_title': 'TF ChIP-seq', 'assembly': 'GRCh38'}
{'biobiosample_summary': 'K562', 'assay_title': 'TF ChIP-seq', 'assembly': 'GRCh38'}
